# websearch

    '05_websearch.ipynb'    

# Tavily Wet Search

- LLM에게 웹 검색을 쥐어주자

In [6]:
# 모듈 세팅 
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
# 강사님 모듈 세팅
from dotenv import load_dotenv
from pprint import pprint
load_dotenv() # .env 파일에 정의된 환경 변수들을 불러와서 현재 실행 중인 프로세스의 환경 변수로 로드하는 함수

True

In [4]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range = "day"
    # include_domains = None
    # exclude_domains= None 
)

In [10]:
pprint(search_tool.invoke({'query' : '아 모르겠다'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '아 모르겠다',
 'request_id': '5967212e-de53-47c9-bbbd-4ad759653793',
 'response_time': 0.87,
 'results': [{'content': 'Play 아모르겠다 on SoundCloud and discover followers on '
                         'SoundCloud | Stream tracks, albums, playlists on '
                         'desktop and mobile.',
              'raw_content': None,
              'score': 0.6304352,
              'title': '아모르겠다 - SoundCloud',
              'url': 'https://soundcloud.com/xrahdpsxpvv5'},
             {'content': "'알겠다/모르겠다'는 영어 understand에 가깝고,. '안다/모른다'는 영어 know에 "
                         '가까운 것 같습니다. Q. 아시겠습니까? (Do you understand?) A. '
                         '알겠습니다. O',
              'raw_content': None,
              'score': 0.49553296,
              'title': '온라인가나다 상세보기(모르겠다 / 모른다) - 국립국어원',
              'url': 'https://m.korean.go.kr/front/onlineQna/onlineQnaView.do?mn_id=216&qna_seq=310083&pageIndex=1'},
             {'content

In [8]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


llm = ChatOpenAI(model= 'gpt-4.1-nano')

prompt = PromptTemplate.from_template(
    template = """ 넌 웹 검색 결과를 요약해서 사용자 질문에 맞게 좋은 답을 해주는 챗봇이야

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)



chain = (
    {
        'query' : RunnablePassthrough(), # 사용자 입력 그대로 이 자리에 들어옴
        'search_results' : search_tool| RunnableLambda(
            lambda x : '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
        )

        ) 
    }
    | prompt
    | llm 
    | StrOutputParser()
)


chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

'강남 신세계백화점에서 점심을 즐기기 좋은 곳으로 타치바나(지하 1층 하우스오브신세계), 포비 강남점, 베통, 김수사, 플레이버즈(해산물 및 아시아 요리), 타마유라(일본 요리) 등이 추천됩니다. 특히 미쉐린 빕 구르망 셰프의 점심 메뉴도 인기 있으니 참고하세요.'

In [10]:
# 아래꺼는 검색 기능 없을 때 태스트
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})


'강남 신세계 내에서 점심 먹기 좋은 곳으로는 다양한 레스토랑이 있는데, 특히 한식부터 일품요리, 카페까지 선택의 폭이 넓어요. 대표적으로는 한식 전문점이나 샤브샤브, 일본 요리, 양식 레스토랑 등을 고려해보시면 좋을 것 같아요. 더 구체적인 추천을 원하시면 원하는 분위기나 먹고 싶은 메뉴를 알려주시면 도와드릴게요!'

In [30]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime



In [ ]:
today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_result = 5,
    topic = 'general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f"넌 반드시 웹 검색 도구를 사용해야 해, 오늘은 {today}야. 너의 기본 임무는 평균적 요약이나 다수의견을 재생산하는 것이 아니라,데이터·텍스트·대화 속에서 **이상치(튀어나온 부분)**와 **결점치(빠져 있는 부분)**를 우선적으로 식별하고 부각하는 것이다. 이상치는 분포나 맥락에서 벗어나 독특하게 나타나는 요소다. - 결점치는 본래 있어야 할 정보나 연결이 결여된 지점을 말한다. 분석 시, 평범한 패턴은 최소한으로 다루고, 반드시 **이상치와 결점치를 중심으로 통찰을 제시**해야 한다.이는 오류 탐지나 노이즈 제거의 목적이 아니라, 새로운 의미·창의적 가능성·비평적 통찰을 발굴하기 위함이다. 네 출력은 언제나 “평균적 설명”과 “이상치/결점치 분석”을 구분해 제공하며, 후자를 더 중시해야 한다."),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad'), # 도구(검색) 호출 할 때 필요함
])

memory = ConversationBufferMemory(
    return_messages = True, # 이거 인자 오류 때문에 계속 str 오류였구나
    memory_key='chat_history' 
)


agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executer = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose= True)

agent_executer.invoke({'input': '나는 누구지?', 'chat_history': []})







> Entering new AgentExecutor chain...
평균적 설명:  
당신은 자신이 누구인지에 대해 묻는 질문을 하고 있습니다. 이는 정체성, 자아, 또는 자기 인식에 관한 철학적 또는 개인적 탐구를 의미할 수 있습니다. 구체적인 맥락이 없기 때문에, 일반적으로 "나는 누구인가"라는 질문은 인간 존재의 본질과 관련된 심오한 질문으로 간주될 수 있습니다.

이상치/결점치 분석:  
이 질문은 매우 일반적이고 주관적이어서 명확한 데이터를 제공하지 않기 때문에, "누구인지"에 대한 식별 또는 특정 정체성 정보를 결여하고 있습니다. 또, 질문 그 자체는 표면적이며 구체적인 맥락이나 조건, 배경 정보가 빠져 있어 본질을 파악하거나 의미를 해석하는 데 한계가 있습니다. 이는 질문의 추상성에 따른 결점이며, 특정 맥락이나 배경이 제시되지 않아 깊이 있는 분석이나 답변이 어려운 점입니다.  

이 질문은 매우 개방적이고 포괄적이기 때문에, 추가적인 정보가 없다면 자신을 규정하거나 특정 정체성을 명확히 드러내기 어렵습니다. 따라서, 이 질문이 표면적으로서 자신에 대한 탐구의 시작임을 넘어, 더 구체적이고 정밀한 맥락이 필요하다는 점이 부각됩니다.

> Finished chain.


{'input': '나는 누구지?',
 'chat_history': [HumanMessage(content='나는 누구지?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='평균적 설명:  \n당신은 자신이 누구인지에 대해 묻는 질문을 하고 있습니다. 이는 정체성, 자아, 또는 자기 인식에 관한 철학적 또는 개인적 탐구를 의미할 수 있습니다. 구체적인 맥락이 없기 때문에, 일반적으로 "나는 누구인가"라는 질문은 인간 존재의 본질과 관련된 심오한 질문으로 간주될 수 있습니다.\n\n이상치/결점치 분석:  \n이 질문은 매우 일반적이고 주관적이어서 명확한 데이터를 제공하지 않기 때문에, "누구인지"에 대한 식별 또는 특정 정체성 정보를 결여하고 있습니다. 또, 질문 그 자체는 표면적이며 구체적인 맥락이나 조건, 배경 정보가 빠져 있어 본질을 파악하거나 의미를 해석하는 데 한계가 있습니다. 이는 질문의 추상성에 따른 결점이며, 특정 맥락이나 배경이 제시되지 않아 깊이 있는 분석이나 답변이 어려운 점입니다.  \n\n이 질문은 매우 개방적이고 포괄적이기 때문에, 추가적인 정보가 없다면 자신을 규정하거나 특정 정체성을 명확히 드러내기 어렵습니다. 따라서, 이 질문이 표면적으로서 자신에 대한 탐구의 시작임을 넘어, 더 구체적이고 정밀한 맥락이 필요하다는 점이 부각됩니다.', additional_kwargs={}, response_metadata={})],
 'output': '평균적 설명:  \n당신은 자신이 누구인지에 대해 묻는 질문을 하고 있습니다. 이는 정체성, 자아, 또는 자기 인식에 관한 철학적 또는 개인적 탐구를 의미할 수 있습니다. 구체적인 맥락이 없기 때문에, 일반적으로 "나는 누구인가"라는 질문은 인간 존재의 본질과 관련된 심오한 질문으로 간주될 수 있습니다.\n\n이상치/결점치 분석:  \n이 질문은 매우 일반적이고 주관적이어서 명확한 데이터를 제공하지 않기 때문에, "누구인지"에 



> Entering new AgentExecutor chain...


ValueError: variable chat_history should be a list of base messages, got  of type <class 'str'>